In [15]:
import requests
import json
import os
from rdflib import Graph, Namespace, Literal, BNode, URIRef
from rdflib.namespace import RDF
import glob
import urllib.parse as up
from graphdbfunctions import *

In [16]:
GRAPHDB_HOST = "http://localhost:7200"
GRAPHDB_REPO = 'gentilly_cadastre' # str | The repository ID
TTL_PATH = "C:/Users/stual/PycharmProjects/peuplement_ontology/ontologie-peuplement/data/rdf"
RDF_PATH_ROOT = "C:/Users/stual/PycharmProjects/peuplement_ontology/ontologie-peuplement/data/rdf-graph-construct"
ONTOLOGY_PATH = "D:/Documents/ONTOLOGY/"
ONTOLOGY_MODULES = ['ontology-adresse.ttl','ontology-cadastre.ttl']
NAMED_GRAPHS = ["http://data.ign.fr/sources/",
        "http://data.ign.fr/plots/frommaps/",
        "http://data.ign.fr/plots/fromregisters/",
        "http://data.ign.fr/ontology",
        "http://data.ign.fr/otherslandmarks"]

In [27]:
MAIN_URI = "http://rdf.geohistoricaldata.org"
MAPS_REGISTERS_LINKS_GRAPH = MAIN_URI + "/id/test"
TMP_GRAPH = MAIN_URI + "/id/temporaire"
ORDRE_GRAPH = MAIN_URI + "/id/ordre"
EVENTS_CHANGES = MAIN_URI + "/id/events_changes"
GRAPHS = [MAPS_REGISTERS_LINKS_GRAPH, TMP_GRAPH, ORDRE_GRAPH]
PRINT = True

In [18]:
remove_named_graphs(GRAPHDB_HOST,GRAPHDB_REPO,GRAPHS)

In [19]:
prefixes = """PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://rdf.geohistoricaldata.org/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://rdf.geohistoricaldata.org/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://rdf.geohistoricaldata.org/id/source/>
PREFIX srctype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/eventType/>"""

### 1. Create similarity links between "maps" plots and "registers" plots

In [20]:
QUERY = prefixes + """

INSERT {
        GRAPH <""" + MAPS_REGISTERS_LINKS_GRAPH + """> {
            ?registersLandmark add:isSimilarTo ?mapsLandmark.
            ?mapsLandmark add:isSimilarTo ?registersLandmark.}}
    WHERE {
        GRAPH <http://rdf.geohistoricaldata.org/plots/frommaps/> {
           ?mapsLandmark a add:Landmark ; add:isLandmarkType cad_ltype:Plot.
        }
        ?registersLandmark a add:Landmark ; add:isLandmarkType cad_ltype:Plot.
        ?registersLandmark add:hasAttribute ?attr1.
        ?attr1 add:isAttributeType cad_atype:PlotMention.
    
        MINUS {?registersLandmark add:isSimilarTo ?mapsLandmark}
        MINUS {?mapsLandmark add:isSimilarTo ?registersLandmark}
        ?mapsLandmark dcterms:identifier ?plotidm.
        ?registersLandmark dcterms:identifier ?plotidr.
        BIND(
             IF(STRENDS(STR(?plotidr), "p"), 
                SUBSTR(STR(?plotidr), 1, STRLEN(STR(?plotidr)) - 1), 
                ?plotidr
              ) AS ?plotid
            )
        FILTER (?plotidm = ?plotid)
}
"""
if PRINT:
  print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>

INSERT {
        GRAPH <http://data.ign.fr/id/test> {
            ?registersLandmark add:isSimilarTo ?mapsLandmark.
            ?mapsLandmark add:isSimilarTo ?re

In [21]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 


### 2. Calcul des écarts relatifs (en jour) entre les états de parcelles

In [22]:
QUERY = """
#Export des écarts temporels entre les états des parcelles issus des matrices
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ofn: <http://www.ontotext.com/sparql/functions/>

CONSTRUCT{?mapsLandmark add:hasTimeGap [ add:hasValue ?ecart; add:isFirstRL ?registerLandmark; add:isSecondRL ?registerLandmark2]}
    WHERE {
    GRAPH <http://data.ign.fr/id/test> 
    {?mapsLandmark add:isSimilarTo ?registerLandmark.
    ?mapsLandmark add:isSimilarTo ?registerLandmark2.}
    ?registerLandmark add:hasTime/add:hasEnd/add:timeStamp ?fin.
    ?registerLandmark2 add:hasTime/add:hasBeginning/add:timeStamp ?debut2 .
BIND(ofn:asDays(?debut2 - ?fin) as ?ecart).
FILTER ((?ecart > 0) && !(sameTerm(?registerLandmark,?registerLandmark2)))
}
"""
if PRINT:
    print(QUERY)


#Export des écarts temporels entre les états des parcelles issus des matrices
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ofn: <http://www.ontotext.com/sparql/functions/>

CONSTRUCT{?mapsLandmark add:hasTimeGap [ add:hasValue ?ecart; add:isFirstRL ?registerLandmark; add:isSecondRL ?registerLandmark2]}
    WHERE {
    GRAPH <http://data.ign.fr/id/test> 
    {?mapsLandmark add:isSimilarTo ?registerLandmark.
    ?mapsLandmark add:isSimilarTo ?registerLandmark2.}
    ?registerLandmark add:hasTime/add:hasEnd/add:timeStamp ?fin.
    ?registerLandmark2 add:hasTime/add:hasBeginning/add:timeStamp ?debut2 .
BIND(ofn:asDays(?debut2 - ?fin) as ?ecart).
FILTER ((?ecart > 0) && !(sameTerm(?registerLandmark,?registerLandmark2)))
}



In [23]:
NAMED_GRAPH = TMP_GRAPH
RDF_PATH = RDF_PATH_ROOT + "/query-result-1.ttl"
load_ttl_file_into_named_graph(GRAPHDB_HOST,GRAPHDB_REPO,NAMED_GRAPH,RDF_PATH)

### 3. Création des relations before et after

In [24]:
QUERY = """
#Calcul des relations d'ordre temporel relatif entre états de parcelles issus des matrices
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ofn: <http://www.ontotext.com/sparql/functions/>

CONSTRUCT {?registerLandmark add:before ?registerLandmark2. ?registerLandmark2 add:after ?registerLandmark.}
    WHERE {
    GRAPH <http://data.ign.fr/id/temporaire> 
    {?mapsLandmark add:hasTimeGap ?gap.
    ?gap add:hasValue ?ecart.
    ?gap add:isFirstRL ?registerLandmark.
    ?gap add:isSecondRL ?registerLandmark2.
    FILTER (!sameTerm(?registerLandmark, ?registerLandmark2))
    }} 
GROUP BY ?mapsLandmark ?registerLandmark ?registerLandmark2

   """
if PRINT:
    print(QUERY)


#Calcul des relations d'ordre temporel relatif entre états de parcelles issus des matrices
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ofn: <http://www.ontotext.com/sparql/functions/>

CONSTRUCT {?registerLandmark add:before ?registerLandmark2. ?registerLandmark2 add:after ?registerLandmark.}
    WHERE {
    GRAPH <http://data.ign.fr/id/temporaire> 
    {?mapsLandmark add:hasTimeGap ?gap.
    ?gap add:hasValue ?ecart.
    ?gap add:isFirstRL ?registerLandmark.
    ?gap add:isSecondRL ?registerLandmark2.
    FILTER (!sameTerm(?registerLandmark, ?registerLandmark2))
    }} 
GROUP BY ?mapsLandmark ?registerLandmark ?registerLandmark2

   


In [25]:
NAMED_GRAPH = ORDRE_GRAPH
RDF_PATH = RDF_PATH_ROOT + "/query-result-2.ttl"
load_ttl_file_into_named_graph(GRAPHDB_HOST,GRAPHDB_REPO,NAMED_GRAPH,RDF_PATH)

### 4. Création des changements et des évènements

In [26]:
QUERY = """
PREFIX cad_etype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/eventType/>
PREFIX cad_ltype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/landmarkType/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/attributeType/>
PREFIX cad: <http://rdf.geohistoricaldata.org/def/cadastre#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX srctype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/sourceType/>
PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
PREFIX time: <http://www.w3.org/2006/time#>
construct {
?registerLandmark add:changedBy ?change.
?change a add:Change.
?change add:appliedTo ?registerLandmark.
?change add:isChangeType ctype:LandmarkDisappearance.
?change add:dependsOn ?event.
?nextPlot add:changedBy ?change2.
?change2 a add:Change.
?change2 add:appliedTo ?nextPlot.
?change2 add:isChangeType ctype:LandmarkAppearance.
?change2 add:dependsOn ?event.
?event a add:Event.
?event cad:isEventType cad_etype:Split.
?event add:hasTime [ a add:TimeInstant; add:timeStamp ?datefin; add:timeCalendar time:Gregorian;
add:timePrecision time:Year].
}
WHERE{
select ?registerLandmark ?nextPlot (IRI(CONCAT("http://rdf.geohistoricaldata.org/id/change/", STRUUID())) AS ?change) (IRI(CONCAT("http://rdf.geohistoricaldata.org/id/change/", STRUUID())) AS ?change2) (IRI(CONCAT("http://rdf.geohistoricaldata.org/id/change/", STRUUID())) AS ?event)
where {
    ?registerLandmark a add:Landmark.
    ?registerLandmark add:hasTime/add:hasEnd/add:timeStamp ?datefin.
    ?registerLandmark add:isLandmarkType cad_ltype:Plot. 
    ?registerLandmark add:hasAttribute ?attrMention.
    ?attrMention add:isAttributeType cad_atype:PlotMention.
    ?attrMention add:hasAttributeVersion/cad:passedTo ?portea.
    ?portea cad:isSourceType srctype:FolioNonBati.
    graph <http://rdf.geohistoricaldata.org/id/ordre>
    {OPTIONAL{?registerLandmark add:after ?nextPlot. }}  
}
GROUP BY ?registerLandmark ?nextPlot
HAVING ((count(?portea)) > 1)
}
"""
if PRINT:
    print(QUERY)


PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
PREFIX time: <http://www.w3.org/2006/time#>
construct {
?registerLandmark add:changedBy ?change.
?change a add:Change.
?change add:appliedTo ?registerLandmark.
?change add:isChangeType ctype:LandmarkDisappearance.
?change add:dependsOn ?event.
?nextPlot add:changedBy ?change2.
?change2 a add:Change.
?change2 add:appliedTo ?nextPlot.
?change2 add:isChangeType ctype:LandmarkAppearance.
?change2 add:dependsOn ?event.
?event a add:Event.
?e

In [28]:
NAMED_GRAPH = EVENTS_CHANGES
RDF_PATH = RDF_PATH_ROOT + "/query-result-3.ttl"
load_ttl_file_into_named_graph(GRAPHDB_HOST,GRAPHDB_REPO,NAMED_GRAPH,RDF_PATH)

### 5. Casser les liens entre les parcelles des registres qui suivent immédiatement une division et les parcelles de cartes

In [ ]:
QUERY = """
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/Type/>

#Récupérer toutes les parcelles qui sont add:after une nouvelle parcelle
#Rattacher ces parcelles à la nouvelle parcelle
construct {
	?newPlot add:isSimilarTo ?registersLandmark2.
    ?registersLandmark2 add:isSimilarTo ?newPlot.
}
where { 
     ?event a add:Event.
     ?event cad:isEventType cad_etype:Split.
     ?change add:dependsOn ?event.
     ?change add:isChangeType ctype:LandmarkAppearance.
     ?change add:appliedTo ?newPlot.
    
	 ?registersLandmark2 a add:Landmark.
     ?registersLandmark2 add:isLandmarkType cad_ltype:Plot.
     ?registersLandmark2 add:hasAttribute ?attr.
     ?attr add:isAttributeType cad_atype:PlotMention.

     ?registersLandmark2 (add:after*) ?newPlot.

     ?change2 add:dependsOn ?event.
     ?change2 add:isChangeType ctype:LandmarkDisappearance.
     ?change2 add:appliedTo ?formerPlot.

     ?registersLandmark2 (add:before*) ?oldPlot.
}
"""